In [ ]:
pip install langchain openai faiss-cpu pypdf groq langchain_community phidata sentence-transformers


: 

In [ ]:
# wget https://www.cyrilshroff.com/wp-content/uploads/2020/09/Guide-to-Litigation-in-India.pdf

: 

In [ ]:
# !wget https://kb.icai.org/pdfs/PDFFile5b28c9ce64e524.54675199.pdf

--2025-03-05 10:37:14--  https://kb.icai.org/pdfs/PDFFile5b28c9ce64e524.54675199.pdf
Resolving kb.icai.org (kb.icai.org)... 54.169.194.86
Connecting to kb.icai.org (kb.icai.org)|54.169.194.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 913275 (892K) [application/pdf]
Saving to: ‘PDFFile5b28c9ce64e524.54675199.pdf.1’

PDFFile5b28c9ce64e5 100%[===================>] 891.87K   265KB/s    in 3.4s    

2025-03-05 10:37:18 (265 KB/s) - ‘PDFFile5b28c9ce64e524.54675199.pdf.1’ saved [913275/913275]



In [3]:
import os

# Set PHI API Key
os.environ["PHI_API_KEY"] = "phi-3bQDo9NUNXEXHuiNA1EQFA5OatR2ZLtlWpuzB_PYNvg"
# Set GROQ API Key
os.environ["GROQ_API_KEY"] = "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"

In [4]:
# Initialize Groq Client
import groq

groq_client = groq.Client(api_key=os.environ["GROQ_API_KEY"])

In [9]:
import os
import openai
import groq
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from phi.agent import Agent
from phi.model.groq import Groq

# Set API keys
# groq.api_key = "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"

# Load PDFs & Process Documents
pdf_files = ["pdfs/Guide_to_Litigation.pdf", "pdfs/Legal_Compliance.pdf"]
documents = []


for pdf in pdf_files:
    loader = PyPDFLoader(pdf)
    documents.extend(loader.load())

# Split text for retrieval
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

# Hugging Face Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# embedding_model = HuggingFaceEmbeddings(model_name="nlpaueb/legal-bert-base-uncased")

# Create FAISS vector database
vector_db = FAISS.from_documents(texts, embedding_model)



C:\Users\Kaust\AppData\Local\Temp\ipykernel_24636\1831819565.py:28: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [ ]:
def retrieve_legal_text(query: str):
    """Fetch relevant legal sections from FAISS DB."""
    docs = vector_db.similarity_search(query, k=2)
    return "\n\n".join([doc.page_content for doc in docs])

def summarize_legal_text(text: str):
    """Summarizes legal text into easy-to-understand language."""
    prompt = f"Summarize the following legal information in simple terms:\n\n{text}"
    return groq.ChatCompletion.create(model="mixtral", messages=[{"role": "user", "content": prompt}])["choices"][0]["message"]["content"]



In [ ]:
query_agent = Agent(
    name="QueryAgent",
    role="Legal Information Retriever",
    model=Groq(id="llama3-8b-8192"),
    tools=[retrieve_legal_text],  # Using tool properly
    instructions="Retrieve the most relevant legal text for a given user query from the FAISS vector database."
)


In [ ]:
summarization_agent = Agent(
    name="SummarizationAgent",
    role="Legal Text Simplifier",
    model=Groq(id="llama3-8b-8192"),
    tools=[summarize_legal_text],  # Uses tool
    instructions="Summarize retrieved legal text into simple terms while maintaining accuracy."
)


In [ ]:
multi_ai_agent=Agent(
    team=[query_agent, summarization_agent],
    name="MultiAgent",
    model=Groq(id="llama3-8b-8192"),
    instructions="Please help the user to retrieve necessary user query related data from database and summarize the legal text into simpler language",
    show_tool_calls=True,
    markdown=True
)

In [ ]:
# multi_ai_agent.print_response("What are the steps involved in filing a lawsuit in India?",stream=True)


In [ ]:
# Multi-Agent Chatbot with Context
class MultiAgentChatbot:
    def __init__(self):
        self.chat_history = []

        self.multi_ai_agent = Agent(
            team=[query_agent, summarization_agent],
            name="MultiAgent",
            model=Groq(id="llama3-8b-8192"),
            instructions="Assist the user with legal queries by retrieving relevant legal sections and summarizing them in simpler terms.",
            show_tool_calls=True,
            markdown=True
        )

    def chat(self, user_input):
        """Process user query with multi-agent system and maintain chat history."""
        self.chat_history.append({"role": "user", "content": user_input})

        response = groq_client.chat.completions.create(
            model="llama3-8b-8192",
            messages=self.chat_history  # Pass entire chat history for context
        )

        bot_response = response.choices[0].message.content
        self.chat_history.append({"role": "assistant", "content": bot_response})

        return bot_response

In [ ]:

chatbot = MultiAgentChatbot()

In [ ]:
# Example Conversation
print(chatbot.chat("What are the steps involved in filing a lawsuit in India?"))


Filing a lawsuit in India involves several steps, which are outlined below:

**Step 1: Determine the appropriate jurisdiction**

* Identify the correct court to file the lawsuit in. In India, the jurisdiction is determined by the location of the defendant.
* In general, a lawsuit can be filed in the court located in the territory where the defendant resides or carries on business.

**Step 2: Identify the type of lawsuit**

* Determine the type of lawsuit to be filed, such as a civil lawsuit (e.g., dispute over a contract, property, or tort) or a criminal lawsuit (e.g., complainant seeking prosecution of the defendant).

**Step 3: Prepare the necessary documents**

* Draft the pleadings, including the plaint (civil lawsuit) or the complaint (criminal lawsuit).
* Prepare supporting documents, such as:
	+ Evidence (e.g., witnesses, exhibits, documents).
	+ Affidavits.
	+ Medical records (if relevant).

**Step 4: Verify the jurisdiction and service**

* Ensure that the lawsuit is filed in 

In [ ]:
print(chatbot.chat("Can you explain step 2 in more detail?"))  # Follow-up with context

Step 2 involves determining the type of lawsuit to be filed. This is an important step because it will influence the legal framework, procedures, and rules that apply to your case.

Here are some factors to consider when determining the type of lawsuit:

**Civil lawsuit:**

* Disputes between individuals, companies, or organizations, such as:
	+ Contract disputes (e.g., breach of contract, non-payment).
	+ Property disputes (e.g., title, possession, liens).
	+ Tort claims (e.g., personal injury, defamation, negligence).
* Claims for damages or relief, such as:
	+ Compensation for harm or loss.
	+ Specific performance (e.g., enforcing an agreement or obligation).
	+ Injunctions (e.g., restraining someone from acting).

**Criminal lawsuit:**

* Matters related to alleged criminal conduct, such as:
	+ Violations of criminal laws (e.g., murder, theft, fraud).
	+ Offenses against the state or society (e.g., mischief, trespass).
* Complaints may be filed by:
	+ Victims of crime (e.g., filing

In [ ]:
print(chatbot.chat("Please explain different levels of judiciary in india"))

In India, the judiciary is divided into three main levels:

**Level 1: District Courts**

* These are the lowest and most local level of courts in India.
* They handle a wide range of cases, including:
	+ Civil disputes (e.g., property, contract, tort).
	+ Criminal cases (e.g., petty crimes, minor offenses).
	+ Family disputes (e.g., divorce, custody, maintenance).
	+ Other types of cases (e.g., labor disputes, revenue disputes).

* District Courts are established in each district of the country and have original jurisdiction over cases within that district.

**Level 2: High Courts**

* These are higher-level courts that have jurisdiction over multiple districts and hear appeals from District Courts.
* High Courts were established under the Government of India Act, 1915, and are now governed by the High Court (Amendment) Act, 1976.
* High Courts have original jurisdiction over certain types of cases, such as:
	+ Constitutional matters (e.g., challenges to Central or State laws).
	+ Wri

In [ ]:
print(chatbot.chat("Under what section of law in India can a man be punished for causing death due to accident?"))

In India, the relevant sections of law that impose criminal penalties for causing death due to an accident depend on the circumstances of the incident. Here are some possible sections of law that may be relevant:

1. Section 304-A of the Indian Penal Code (IPC): This section deals with "Causing death by negligence" and can be charged against a person who, due to his own conduct, is responsible for the death of another person. The prosecution must show that the accused person had a duty to prevent the death, failed to perform that duty, and that the death was a direct result of that failure.
2. Section 279 of the Indian Penal Code (IPC): This section deals with "Rash driving or riding on a public way" and can be charged against a person who drives a vehicle recklessly and causes an accident that results in death.
3. Section 304-B of the Indian Penal Code (IPC): This section deals with "Dowry death" and can be charged against a person who causes the death of a married woman due to dowry-

In [ ]:
# # Install required packages
# !pip install streamlit pyngrok langchain openai faiss-cpu pypdf groq
# !pip install langchain openai faiss-cpu pypdf groq langchain_community phidata

# # Write app.py
# with open("app.py", "w") as f:
#     f.write('''import streamlit as st
# import os
# import groq
# from langchain.document_loaders import PyPDFLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import FAISS
# from phi.agent import Agent
# from phi.model.groq import Groq

# # Set Groq API key
# os.environ["GROQ_API_KEY"] = "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"
# groq_client = groq.Groq()

# # Initialize vector database
# def initialize_vector_db():
#     # Download PDFs
#     pdf_files = [
#         "https://www.cyrilshroff.com/wp-content/uploads/2020/09/Guide-to-Litigation-in-India.pdf",
#         "https://kb.icai.org/pdfs/PDFFile5b28c9ce64e524.54675199.pdf"
#     ]

#     # Load PDFs
#     documents = []
#     for url in pdf_files:
#         loader = PyPDFLoader(url)
#         documents.extend(loader.load())

#     # Split text for retrieval
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
#     texts = text_splitter.split_documents(documents)

#     # Create FAISS vector database
#     embedding_model = HuggingFaceEmbeddings(model_name="nlpaueb/legal-bert-base-uncased")
#     return FAISS.from_documents(texts, embedding_model)

# # Initialize agents
# vector_db = initialize_vector_db()

# def retrieve_legal_text(query: str):
#     """Fetch relevant legal sections from FAISS DB."""
#     docs = vector_db.similarity_search(query, k=2)
#     return "<br>".join([doc.page_content for doc in docs])

# def summarize_legal_text(text: str):
#     """Summarizes legal text into easy-to-understand language."""
#     prompt = f"Summarize the following legal information in simple terms:<br><br>{text}"
#     return groq.ChatCompletion.create(
#         model="mixtral",
#         messages=[{"role": "user", "content": prompt}]
#     )["choices"][0]["message"]["content"]

# query_agent = Agent(
#     name="QueryAgent",
#     role="Legal Information Retriever",
#     model=Groq(id="llama3-8b-8192"),
#     tools=[retrieve_legal_text],
#     instructions="Retrieve the most relevant legal text for a given user query from the FAISS vector database.",
#     show_tool_calls=True,
#     markdown=True
# )

# summarization_agent = Agent(
#     name="SummarizationAgent",
#     role="Legal Text Simplifier",
#     model=Groq(id="llama3-8b-8192"),
#     tools=[summarize_legal_text],
#     instructions="Summarize retrieved legal text into simple terms while maintaining accuracy.",
#     show_tool_calls=True,
#     markdown=True
# )

# multi_ai_agent = Agent(
#     team=[query_agent, summarization_agent],
#     name="MultiAgent",
#     model=Groq(id="llama3-8b-8192"),
#     instructions="Assist the user with legal queries by retrieving relevant legal sections and summarizing them in simpler terms.",
#     show_tool_calls=True,
#     markdown=True
# )

# # Create the main interface
# def main():
#     st.title("Legal Information Retrieval System")
#     st.subheader("Search and understand legal documents")

#     # Query input
#     query = st.text_area(
#         "Enter your legal query:",
#         height=100,
#         placeholder="Type your question about legal procedures or concepts..."
#     )

#     if st.button("Search"):
#         # Process the query
#         response = multi_ai_agent.chat(query)

#         # Display results
#         st.subheader("Results")
#         st.write(response)

# if __name__ == "__main__":
#     main()''')

# # Start Streamlit App in the Background
# !nohup streamlit run app.py --server.port 8501 >> /abcd.txt 2>&1 &
# # Connect with ngrok
# !ssh -o StrictHostKeyChecking=no -R 80:localhost:8501 nokey@localhost.run

In [14]:
%%writefile main.py
import os
import groq
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from phi.agent import Agent
from phi.model.groq import Groq

# **🔹 Set API Keys**
os.environ["PHI_API_KEY"] = "phi-3bQDo9NUNXEXHuiNA1EQFA5OatR2ZLtlWpuzB_PYNvg"
os.environ["GROQ_API_KEY"] = "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"

# **🔹 Initialize Groq Client**
groq_client = groq.Client(api_key=os.environ["GROQ_API_KEY"])

# **🔹 Load All PDFs from 'pdfs/' Directory**
pdf_folder = "/content/pdfs"
pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.endswith(".pdf")]

documents = []
for pdf in pdf_files:
    loader = PyPDFLoader(pdf)
    documents.extend(loader.load())

# **🔹 Split Text for Retrieval**
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

# **🔹 Hugging Face Embeddings**
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# **🔹 Create FAISS Vector Store**
vector_db = FAISS.from_documents(texts, embedding_model)

# **🔹 Function to Retrieve Legal Text**
def retrieve_legal_text(query: str):
    """Fetch relevant legal sections from FAISS DB."""
    docs = vector_db.similarity_search(query, k=2)
    return "\n\n".join([doc.page_content for doc in docs])

# **🔹 Function to Summarize Legal Text**
def summarize_legal_text(text: str):
    """Summarizes legal text into easy-to-understand language."""
    prompt = f"Summarize the following legal information in simple terms:\n\n{text}"
    return groq_client.chat.completions.create(
        model="mixtral",
        messages=[{"role": "user", "content": prompt}]
    )["choices"][0]["message"]["content"]

# **🔹 Query Agent**
query_agent = Agent(
    name="QueryAgent",
    role="Legal Information Retriever",
    model=Groq(id="llama3-8b-8192"),
    tools=[retrieve_legal_text],
    instructions="Retrieve the most relevant legal text for a given user query from the FAISS vector database."
)

# **🔹 Summarization Agent**
summarization_agent = Agent(
    name="SummarizationAgent",
    role="Legal Text Simplifier",
    model=Groq(id="llama3-8b-8192"),
    tools=[summarize_legal_text],
    instructions="Summarize retrieved legal text into simple terms while maintaining accuracy."
)

# **🔹 Multi-Agent System**
multi_ai_agent = Agent(
    team=[query_agent, summarization_agent],
    name="MultiAgent",
    model=Groq(id="llama3-8b-8192"),
    instructions="Retrieve legal data and summarize it for users.",
    show_tool_calls=True,
    markdown=True
)

# **🔹 Multi-Agent Chatbot**
class MultiAgentChatbot:
    def __init__(self):
        self.chat_history = []
        self.multi_ai_agent = multi_ai_agent

    def chat(self, user_input):
        """Process user query with multi-agent system and maintain chat history."""
        self.chat_history.append({"role": "user", "content": user_input})
        response = groq_client.chat.completions.create(
            model="llama3-8b-8192",
            messages=self.chat_history
        )
        bot_response = response.choices[0].message.content
        self.chat_history.append({"role": "assistant", "content": bot_response})
        return bot_response

# **🔹 Initialize Chatbot**
chatbot = MultiAgentChatbot()


Overwriting main.py


In [15]:
%%writefile app.py
import streamlit as st
from main import chatbot  # Import chatbot instance

# **🔹 Streamlit UI**
st.title("⚖️ AI-Powered Legal Chatbot")
st.write("Ask legal questions and get AI-generated responses based on stored legal documents.")

# **🔹 User Input Box**
user_input = st.text_input("Enter your legal query:")

# **🔹 Process Query on Button Click**
if st.button("Submit"):
    if user_input:
        with st.spinner("Fetching answer..."):
            response = chatbot.chat(user_input)
        st.subheader("Response:")
        st.write(response)
    else:
        st.warning("Please enter a question before submitting.")

# **🔹 Footer**
st.markdown("---")
st.write("Powered by **Phi Agents, Groq AI, and LangChain**")


Overwriting app.py


In [20]:

# ✅ Start Streamlit App in the Background
!nohup streamlit run app.py --server.port 8503 >> /abcd.txt 2>&1 &

# ✅ Connect with ngrok
# from pyngrok import ngrok
# import time

# time.sleep(3)  # Give Streamlit time to start
# public_url = ngrok.connect(8501)
# print(f"🚀 Open your RAG chatbot app here: {public_url}")

!ssh -o StrictHostKeyChecking=no -R 80:localhost:8503 nokey@localhost.run


Welcome to localhost.run!

Follow your favourite reverse tunnel at [https://twitter.com/localhost_run].

To set up and manage custom domains go to https://admin.localhost.run/

More details on custom domains (and how to enable subdomains of your custom
domain) at https://localhost.run/docs/custom-domains

If you get a permission denied error check the faq for how to connect with a key or
create a free tunnel without a key at [http://localhost:3000/docs/faq#generating-an-ssh-key].

To explore using localhost.run visit the documentation site:
https://localhost.run/docs/


** your connection id is d9d1005b-0da9-4b96-8b01-11ecdc6a5af2, please mention it if you send me a message about an issue. **

authenticated as anonymous user
Connection to localhost.run closed.
